# Energy Flows for PV ICE

This journal documents and demonstrates the new Energy flows calculation capacity of PV ICE

In [1]:
import os
from pathlib import Path
import PV_ICE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP')

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\PV_ICE\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\PV_ICE\PV_ICE\TEMP


In [2]:
PV_ICE.__version__

'v0.2.0+146.g1c34842'

### Add Scenarios and Materials


In [3]:
cwd=os.getcwd()
print(os.getcwd())

C:\Users\sayala\Documents\GitHub\PV_ICE\docs\tutorials


In [4]:
MATERIALS = ['glass']#,'aluminium_frames','silver','silicon', 'copper', 'encapsulant', 'backsheet']
MATERIAL = MATERIALS[0]
moduleFile_recycle = r'..\baselines\perovskite_modules_US_recycle.csv'
moduleFile_reMFG = r'..\baselines\perovskite_modules_US_reMFG.csv'

In [5]:
r1 = PV_ICE.Simulation(name='perovskite_energies', path=testfolder)

r1.createScenario(name='perovskite_recycle', file=moduleFile_recycle)
for mat in range (0, len(MATERIALS)):
    MATERIALBASELINE = r'..\baselines\perovskite_material_'+MATERIALS[mat]+'.csv'
    r1.scenario['perovskite_recycle'].addMaterial(MATERIALS[mat], file=MATERIALBASELINE)

r1.createScenario(name='perovskite_reMFG', file=moduleFile_reMFG)
for mat in range (0, len(MATERIALS)):
    MATERIALBASELINE = r'..\baselines\perovskite_material_'+MATERIALS[mat]+'.csv'
    r1.scenario['perovskite_reMFG'].addMaterial(MATERIALS[mat], file=MATERIALBASELINE)

path = C:\Users\sayala\Documents\GitHub\PV_ICE\PV_ICE\TEMP


### Set All Material Virgin, MFG, and circularity to 0

Do we want to do this?

In [6]:
#r1.scenMod_noCircularity() # sets all module and material circular variables to 0, creating fully linear
#r1.scenMod_PerfectManufacturing() #sets all manufacturing values to 100% efficiency/yield
#check:
#r1.scenario['USHistory'].material['glass'].materialdata['mat_MFG_eff']

### Run the Mass Flow Calculations on All Scenarios and Materials

In [7]:
r1.calculateMassFlow()

Working on Scenario:  perovskite_recycle
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
Working on Scenario:  perovskite_reMFG
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass


## Testing Energy Flows

In [10]:
matEfile_glass = str(Path().resolve().parent.parent /'PV_ICE' / 'baselines'/'perovskite_energy_material_glass.csv')

modEfile = str(Path().resolve().parent.parent /'PV_ICE' / 'baselines'/'perovskite_energy_modules.csv')

In [11]:
file = matEfile_glass
csvdata = open(str(file), 'r', encoding="UTF-8")
csvdata = open(str(file), 'r', encoding="UTF-8-sig")
firstline = csvdata.readline()
secondline = csvdata.readline()



head = firstline.rstrip('\n').split(",")
meta = dict(zip(head, secondline.rstrip('\n').split(",")))



data = pd.read_csv(csvdata, names=head)
data.loc[:, data.columns != 'year'] = data.loc[:, data.columns != 'year'].astype(float)
matEfile_glass_simple = data.copy()

In [13]:
file = modEfile
csvdata = open(str(file), 'r', encoding="UTF-8")
csvdata = open(str(file), 'r', encoding="UTF-8-sig")
firstline = csvdata.readline()
secondline = csvdata.readline()



head = firstline.rstrip('\n').split(",")
meta = dict(zip(head, secondline.rstrip('\n').split(",")))



data = pd.read_csv(csvdata, names=head)
data.loc[:, data.columns != 'year'] = data.loc[:, data.columns != 'year'].astype(float)
modEfile_simple = data.copy()

In [17]:
r1reMFG = r1.calculateEnergyFlow(scenarios='perovskite_reMFG', materials='glass', modEnergy=modEfile_simple, matEnergy=matEfile_glass_simple)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [18]:
r1reMFG

,mod_MFG,mod_Install,mod_OandM,mod_Repair,mod_Demount,mod_Store,mod_Resell_Certify,mod_ReMFG_Disassembly,mod_Recycle_Crush,mat_extraction,mat_MFG,mat_MFGScrap_LQ,mat_MFGScrap_HQ,mat_Landfill,mat_EoL_ReMFG_clean,mat_Recycled_LQ,mat_Recycled_HQ
0,4.495246e+08,1.286290e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.000000,0.0,1.795215e+10,6.103942e+12,5.060085e+10,6.505823e+07,2.439684e+08,0.000000e+00,0.0,0.0
1,4.194213e+08,1.200151e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.000952,0.0,1.671589e+10,5.683509e+12,4.349124e+10,5.591731e+07,2.096899e+08,6.596282e+03,0.0,0.0
2,1.232143e+09,3.525711e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.074076,0.0,4.900701e+10,1.666242e+13,1.168786e+11,1.502724e+08,5.635217e+08,5.134830e+05,0.0,0.0
3,1.197202e+09,3.425728e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.944228,0.0,4.747267e+10,1.614036e+13,9.777793e+10,1.257145e+08,4.714303e+08,6.545214e+06,0.0,0.0
4,9.063789e+08,2.593554e+06,0.0,0.0,5.845726e+08,0.0,0.0,5.846748,0.0,3.586751e+10,1.219449e+13,6.609777e+10,8.498284e+07,3.186916e+08,4.052860e+07,0.0,0.0
5,8.900034e+08,2.546696e+06,0.0,0.0,5.845726e+08,0.0,0.0,24.793302,0.0,3.514629e+10,1.194910e+13,5.714793e+10,7.347591e+07,2.755600e+08,1.718627e+08,0.0,0.0
6,1.239711e+09,3.547366e+06,0.0,0.0,5.845726e+08,0.0,0.0,82.550303,0.0,4.880315e+10,1.659178e+13,6.348167e+10,8.161929e+07,3.061568e+08,5.722237e+08,0.0,0.0
7,1.223267e+09,3.500313e+06,0.0,0.0,5.845726e+08,0.0,0.0,230.773043,0.0,4.804176e+10,1.633267e+13,5.207523e+10,6.695387e+07,2.513130e+08,1.599677e+09,0.0,0.0
8,9.828429e+08,2.812352e+06,0.0,0.0,5.845726e+08,0.0,0.0,564.238855,0.0,3.855502e+10,1.310748e+13,4.179201e+10,5.373258e+07,2.020742e+08,3.911201e+09,0.0,0.0
9,9.725420e+08,2.782876e+06,0.0,0.0,5.845726e+08,0.0,0.0,1241.258663,0.0,3.807075e+10,1.294284e+13,4.126708e+10,5.305767e+07,2.002357e+08,8.604179e+09,0.0,0.0


In [21]:
r1reCYCLE = r1.calculateEnergyFlow(scenarios='perovskite_recycle', materials='glass', modEnergy=modEfile_simple, matEnergy=matEfile_glass_simple)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [22]:
r1reCYCLE

,mod_MFG,mod_Install,mod_OandM,mod_Repair,mod_Demount,mod_Store,mod_Resell_Certify,mod_ReMFG_Disassembly,mod_Recycle_Crush,mat_extraction,mat_MFG,mat_MFGScrap_LQ,mat_MFGScrap_HQ,mat_Landfill,mat_EoL_ReMFG_clean,mat_Recycled_LQ,mat_Recycled_HQ
0,4.495246e+08,1.286290e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,0.000000e+00,1.795215e+10,6.103942e+12,5.060085e+10,6.505823e+07,2.439684e+08,0.0,0.000000e+00,0.000000e+00
1,4.194213e+08,1.200151e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,5.796077e-01,1.671589e+10,5.683509e+12,4.349124e+10,5.591731e+07,2.096900e+08,0.0,1.362511e+04,1.051080e+01
2,1.232143e+09,3.525711e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,4.511916e+01,4.900701e+10,1.666243e+13,1.168786e+11,1.502725e+08,5.635266e+08,0.0,1.060637e+06,8.182057e+02
3,1.197202e+09,3.425728e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,5.751204e+02,4.747278e+10,1.614040e+13,9.777816e+10,1.257148e+08,4.714922e+08,0.0,1.351962e+07,1.042942e+04
4,9.063789e+08,2.593554e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,3.561201e+03,3.586820e+10,1.219473e+13,6.609904e+10,8.498448e+07,3.190745e+08,0.0,8.371481e+07,6.457999e+04
5,8.900034e+08,2.546696e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,1.510137e+04,3.514921e+10,1.195009e+13,5.715269e+10,7.348203e+07,2.771804e+08,0.0,3.549950e+08,2.738533e+05
6,1.239711e+09,3.547366e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,5.028064e+04,4.881282e+10,1.659508e+13,6.349430e+10,8.163553e+07,3.115365e+08,0.0,1.181970e+09,9.118056e+05
7,1.223267e+09,3.500313e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,1.405618e+05,4.806873e+10,1.634188e+13,5.210460e+10,6.699163e+07,2.663238e+08,0.0,3.304250e+09,2.548993e+06
8,9.828429e+08,2.812352e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,3.436728e+05,3.862098e+10,1.313000e+13,4.186382e+10,5.382491e+07,2.387754e+08,0.0,8.078875e+09,6.232275e+06
9,9.725420e+08,2.782876e+06,0.0,0.0,5.845726e+08,0.0,0.0,0.0,7.560394e+05,3.821586e+10,1.299239e+13,4.142506e+10,5.326079e+07,2.809740e+08,0.0,1.777257e+10,1.371027e+07


# Calculations for file prep (module and glass)

### Module
We are doing a case study comparing recycling versus remanufacturing the glass for a perovskite module. The assumption is that the perovskite module lasts for 15 years, with weibull parameters that fail 10% of the modules before year 15, and a power degradation such that the module is at 80% of nameplate power at year 15. See the Lifetime vs Recycling journal for calculation of weibull parameters and degradation rates.

### Glass
The assumption is that a perovskite module will be a glass-glass package. Modern c-Si glass-glass (35% marketshare) bifacial modules (27% marketshare) are most likely 2.5mm front glass (28% marketshare) and 2.5 mm back glass (95% marketshare) [ITRPV 2022]. Therefore, we will assume a perovskite glass glass module will use 2 sheets of glass that are 2.5 mm thick.


In [ ]:
density_glass = 2500*1000 # g/m^3    
glass_thickness = 2.5/1e3 #m
mass_glass = 2*glass_thickness*density_glass
print('The mass of glass per module area for a perovskite glass-glass package is assumed to be '+
     str(mass_glass) + ' g/m^2.')

## Energy flows (module and glass)

### Module
We will assume that a perovskite module is disassemble-able or openable with either a laser applied to a vacuum edge seal OR with a hot knife. Laser edge seal can be done with a 16 W laser. 

The hot knife procedure with EVA heats the blade to 300 C (https://www.npcgroup.net/eng/solarpower/reuse-recycle/dismantling#comp) and is currently only used on glass-backsheet modules. The NPC website indicates that cycle time is 60 seconds for one 6x10 cell module. Small commercially availble hot knives can achieve greater than 300C drawing less than 150W. We will assume worst case scenario; hot knife for 60 seconds at 150 W

In [ ]:
e_hotknife_tot = 150*60*(1/3600)*(1/1000) # 150 W * 60 s = W*s *(hr/s)*(kW/W)
area_mod = 1.090*2.100 #m
e_hotknife = e_hotknife_tot/area_mod # E/module to E/m2
print('Energy for hot knife separation is '+ str(round(e_hotknife, 4))+' kWh/m2. This will be used as energy of module reMFG Disassembly')

### Glass
The process to remove the perovskite from the glass is relatively simple regardless of architecture (all back contact or stack). A dunk in a water-based solvent at room temperature can remove the perovskite layer, and simple heating/baking or UV+Ozone steps should round out the cleaning preparation for a new perovskite deposition. 

Using Rodriguez-Garcia G, Aydin E, De Wolf S, Carlson B, Kellar J, Celik I. Life Cycle Assessment of Coated-Glass Recovery from Perovskite Solar Cells. ACS Sustainable Chem Eng [Internet]. 2021 Nov 3 [cited 2021 Nov 8]; Available from: https://doi.org/10.1021/acssuschemeng.1c05029, we will assume a room temperature water bath with sonication, a heating/drying/baking step, and a UV+Ozone step.


In [ ]:
e_sonicate = 4  #kWh/m2 ultrasonication
e_uvozone = 3.57  #kWh/m2 ozone+UV
e_bake = 0.06  #kWh/m2 hot plate drying
e_glassreMFG_area = e_sonicate+e_uvozone+e_bake
e_glass_reMFG_mass = e_glassreMFG_area/(mass_glass/1000) #kWh/m2 *m2/g

print('Energy to remanufacture the glass (material energy only) is ' + str(round(e_glass_reMFG_mass, 2)) + 'kWh/kg')

